<a href="https://colab.research.google.com/github/grochot/offline_analysis/blob/main/Noise_System_Offline_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Montowanie dysku Google

Dysk montujemy, aby móc wczytywać pliki z Dysku Google

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


#Files path

Ustawia scieżkę do folderu, w którym znajdują się pliki do analizy.

In [ ]:
src_path = '/gdrive/MyDrive/AGH/wisniowski/test_data2/' #data folder path

#Background

Ten kod uruchamiamy. Nie ma potrzeby odsłaniać zawartości. Znajdują się tutaj wszystkie funkcje liczące i wyświetlające dane.

In [ ]:
header_list = ['time (s)', 'V (V)', 'f (Hz)', 'AHac (Oe)', 'Vsense (V)', 'Vbias (V)', 'I (A)', 'Phase', 'R (ohm)', 'dI/dH', "dI", 'dV', 'X field (Oe)', 'Y field (Oe)', 'Z field (Oe)',  'Hset (Oe)', 'G',  'I/Phase', 'I/Ax', 'dG',  "dR", 'dI/dV', 'dV/dI',  'dG/dH', 'NdR', 'NdG', 'NdI', 'HdIS', 'dV/dH', 'SPdI',  'HdIS', 'HdRS', 'HdGS',  "G(t)",  "R(t)",  'index']
raw_data_list = [ 'V (V)',
                    'I (A)',
                    'R (ohm)',
                    'G',
                    'X field (Oe)',
                    'Y field (Oe)',
                    'Z field (Oe)',
                    'Hset (Oe)',
                    'dR/dH',
                    'dG/dH',
                    'dI',
                    'dV',
                    'dI/dH',
                    'dV/dH',
                    'NdI',
                    'SPdI',
                    'HdIS',
                    'HdR',
                    'HdG',
                    'dR',
                    'dG',
                    'NdR',
                    'NdG',
                    'HdRS',
                    'HdGS']
def data_parser():
  import glob
  import os
  import pandas as pd
  data = {}
  all_files = glob.glob(os.path.join(src_path, "*.csv"))
  all_files_enum = enumerate(all_files)
  for fn in all_files_enum:
    dev = os.path.basename(fn[1]).split("_")[0]
    df = pd.read_csv(fn[1], comment='#', low_memory = False)
    df['indx'] = df.index
    data[fn[1]] = df


  dopasowania = {}
  for row in header_list:
    print(row)
  for i in range (0, len(all_files)):
    header =  data[all_files[i]].columns.values
    for head in header:
      if head in header_list:
          continue
      if (head in header_list) or (head in list(dopasowania.keys())):
          data[all_files[i]].rename(columns={head: dopasowania[head]}, inplace=True)
          continue
      else:
          #print(list(dopasowania.values()))
          new_header = input("Enter new header for " + head + " in " + all_files[i] + ": ")
          dopasowania[head] = new_header
          data[all_files[i]].rename(columns={head: new_header}, inplace=True)

    data[all_files[i]].to_csv(all_files[i]+"_headers_ok", index=False)
    print("Created file: " + all_files[i]+"_headers_ok")

In [ ]:
from collections import defaultdict
import glob
import os
import pandas as pd

import numpy as np
from scipy.optimize import curve_fit
from scipy.fft import fft, rfft
from scipy.fft import fftfreq, rfftfreq
import altair as alt
import ipywidgets as widgets
#from altair_data_server import data_server
alt.data_transformers.disable_max_rows()
import logging

logger = logging.getLogger('LOG:')

logging.basicConfig(level=logging.DEBUG)
logger.setLevel(logging.INFO)



time = 'time (s)'
V = 'V (V)'
f = 'f (Hz)'
AHac = 'AHac (Oe)'
Vsense = 'Vsense (V)'
Vbias = 'Vbias (V)'
I = 'I (A)'
Phase = 'Phase'
R = 'R (ohm)'
dI_dH = 'dI/dH'
dI = "dI"
dV = 'dV'
X_field = 'X field (Oe)'
Y_field = 'Y field (Oe)'
Z_field = 'Z field (Oe)'
H_set = 'Hset (Oe)'
G = 'G'
I_phase = 'I/Phase'
I_Ax = 'I/Ax'
dG = 'dG'
dR = "dR"
dI_dV = 'dI/dV'
dV_dI = 'dV/dI'
dG_dH = 'dG/dH'
NdR = 'NdR'
NdG = 'NdG'
NdI = 'NdI'
HdIS = 'HdIS'
dV_dH = 'dV/dH'
SPdI = 'SPdI'
HdIS = 'HdIS'
HdRS = 'HdRS'
HdGS = 'HdGS'
G_t = "G(t)"
R_t = "R(t)"
indx = 'index'
HdR = 'HdR'
HdG = 'HdG'

# Shortcut to the GDrive
#src_path = '/gdrive/MyDrive/AGH/wisniowski/test_data/' #data folder path
def read_files():
  data = {}
  all_files = glob.glob(os.path.join(src_path, "*.csv_headers_ok"))
  all_files_enum = enumerate(all_files)
  for fn in all_files_enum:
    dev = os.path.basename(fn[1]).split("_")[0]
    df = pd.read_csv(fn[1], comment='#', low_memory = False)
    df[indx] = df.index
    data[fn[1]] = df.dropna()

  print(logger.info("Files loaded:"+str(len(all_files))))
  return data, all_files


In [ ]:
############################################Funkcje liczące ############################

#Raw functions
def diffs(x,y):
    dy = np.diff(y)
    dx = np.diff(x)
    res = [i / j for i, j in zip(dy, dx)]
    return res

def diffIV(x):

    dif = np.diff(x)
    return dif

def SlopeDiff(x,y):
    res = []
    for i in range(1,len(x)-1):
        res.append((y[i+1]-y[i-1])/x[i+1]-x[i-1])
    return res

def NormalizedDiff(x):
    max_l = max(x)
    min_l = min(x)
    res = []
    for i in range(1,len(x)-1): ##
        res.append((x[i+1]-x[i-1])/(max_l-min_l))
    return res

def HdIScount(x,R):
    res =[]
    for i in range(len(x)):
        res.append(float(x[i])*float(R[i]))
    return res

def division(x,y):
    res = [i / j for i, j in zip(x, y)]
    return res



#Functions analysis

def NLI(list_v_i, list_H, *args):
  results = []
  for i in range(1,len(list_v_i)-1):
    result = (list_v_i[i+1]-list_v_i[i])-(list_v_i[i]-list_v_i[i-1])
    results.append(result)
  return list_H[1:len(list_v_i)-1], results

def NLV(list_v_i, list_H, *args):
  results = []
  for i in range(1,len(list_v_i)-1):
    result = (list_v_i[i+1]-list_v_i[i])-(list_v_i[i]-list_v_i[i-1])
    results.append(result)
  return list_H[1:len(list_v_i)-1], results

def HSI(list_i, list_H, *args):
  return list_H, list_i

def diffJ(A, list_I, list_H, *args):
  results = []
  for i in range(1,len(list_I)-1):
    result = (list_I[i+1]/A-list_I[i-1]/A)
    results.append(result)
  return list_H[1:len(list_I)-1], results

def diffHDC(list_H, *args):
  results = []
  for i in range(1,len(list_H)-1):
    result = (list_H[i+1]-list_H[i-1])
    results.append(result)
  return list_H[1:len(list_H)-1],results

def NdiffJ(A, list_I, list_H, *args):
  j_n = []
  j = diffJ(A, list_I, list_H)[1]
  j_max = max(j)
  j_min = min(j)
  for i in j:
    if j_max - j_min != 0:
      j_n.append(i/(j_max-j_min))
    else:
      j_n.append(i/(1e-20))
  return list_H[1:len(list_I)-1], j_n

def I_min_max(list_i_v, list_H, *args):
  try:
    h_min = list_H.index(min(list_H))
    h_max = list_H.index(max(list_H))
    h_0 = list_H.index(0)
    return list_i_v[h_min], list_i_v[h_max], list_i_v[h_0]
  except:
    return list_i_v[h_min], list_i_v[h_max]


def V_min_max(list_i_v, list_H, *args):
  try:
    h_min = list_H.index(min(list_H))
    h_max = list_H.index(max(list_H))
    h_0 = list_H.index(0)
    return list_i_v[h_min], list_i_v[h_max], list_i_v[h_0]
  except:
    return list_i_v[h_min], list_i_v[h_max]


def dFitI(list_i_v, list_h, start, stop, *args):
  coef = np.polyfit(list_h[start:stop],list_i_v[start:stop],1)
  print(coef)
  poly1d_fn = np.poly1d(coef)
  (print(poly1d_fn))
  differences = []
  for i in range(len(list_i_v)):
    differences.append(list_i_v[i]-poly1d_fn(list_h[i]))
  return list_h, differences

def dFitV(list_i_v, list_h, start, stop, *args):
  coef = np.polyfit(list_h[start:stop],list_i_v[start:stop],1)
  poly1d_fn = np.poly1d(coef)
  differences = []
  for i in range(len(list_i_v)):
    differences.append(list_i_v[i]-poly1d_fn(list_h[i]))
  return list_h, differences

def dfiI(list_i, list_H, *args):
  results = []
  i_max = max(list_i)
  i_min = min(list_i)
  for i in range(len(list_i)-1):
    if i_max-i_min !=0:
      results.append(180-np.arccos(2*(list_i[i+1]-list_i[i])/(i_max-i_min))*180/np.pi)
    else:
      results.append(180-np.arccos(2*(list_i[i+1]-list_i[i])/(1e-15))*180/np.pi)
  return list_H[1:len(list_i)-1], results

def HSR(list_i, list_H, *args):
  return list_H, list_i   #TO DO


def HSG(list_i, list_H, *args):
  return list_H, list_i     #TO DO

def dFitR(list_r_g, list_h, start, stop, *args):
  coef = np.polyfit(list_h[start:stop],list_r_g[start:stop],1)
  poly1d_fn = np.poly1d(coef)
  differences = []
  for i in range(len(list_r_g)):
    differences.append(list_r_g[i]-poly1d_fn(list_h[i]))
  return list_h, differences                              #TO DO

def dFitG(list_r_g, list_h, start, stop, *args):
  coef = np.polyfit(list_h[start:stop],list_r_g[start:stop],1)
  poly1d_fn = np.poly1d(coef)
  differences = []
  for i in range(len(list_r_g)):
    differences.append(list_r_g[i]-poly1d_fn(list_h[i]))
  return list_h, differences                                # TO DO


def R_min_max_0(list_r_g, list_H, *args):
  try:
    h_min = list_H.index(min(list_H))
    h_max = list_H.index(max(list_H))
    h_0 = list_H.index(0)
    return list_r_g[h_min], list_r_g[h_max], list_r_g[h_0]
  except:
    return list_r_g[h_min], list_r_g[h_max]

def G_min_max_0(list_r_g, list_H, *args):
  try:
    h_min = list_H.index(min(list_H))
    h_max = list_H.index(max(list_H))
    h_0 = list_H.index(0)
    return list_r_g[h_min], list_r_g[h_max], list_r_g[h_0]
  except:
    return list_r_g[h_min], list_r_g[h_max]

def d2R(list_dR_dG, list_dI_V, list_H, *args):
  results = []
  for i in range(len(list_dI_V)):
    results.append(list_dR_dG[i]/list_dI_V[i])
  return list_H, results

def d2G(list_dR_dG, list_dI_V, list_H, *args):
  results = []
  for i in range(len(list_dI_V)):
    try:
      results.append(list_dR_dG[i]/list_dI_V[i])
    except:
      results.append(list_dR_dG[i]/0.000000000001)
  return list_H, results


def FitR(xdata, ydata, start, stop, *args):
  def func(x:list, a:float, b:float, c:float):
    y = float(a) + np.multiply(float(b),x) + np.multiply(float(c),x)
    return y
  popt, pcov = curve_fit(func, xdata[start:stop], ydata[start:stop])
  return xdata, func(xdata, *popt), popt


def FitG(xdata, ydata, start, stop, *args):
  def func(x:list, a:float, b:float, c:float):
    y = float(a) + np.multiply(float(b),x) + np.multiply(float(c),x)
    return y
  popt, pcov = curve_fit(func, xdata[start:stop], ydata[start:stop])
  return xdata, func(xdata, *popt), popt


def TdI(list_time, list_I, list_HAC, *args):
  results = []
  for i in range(1,len(list_time)-1):
    dI = list_I[i+1]-list_I[i-1]
    dt = list_time[i+1] - list_time[i-1]
    results.append(dI/dt)
  return list_HAC[1:len(list_time)-1], results


def FTIt(signal, sampling_rate, *args):
    time_step = 1.0/sampling_rate
    duration = len(signal)/sampling_rate
    time_axis = np.arange(0, duration, time_step)
    frequencies = rfftfreq(len(signal), d = time_step)
    fourier = rfft(signal)
    amplitude = 2*np.abs(fourier)/len(signal)
    return frequencies, amplitude
################### RAW FUNCTION ##################

def V_raw(v_data, *args):
    return v_data

def R_raw(v_data, i_data, *args):
    return division(v_data, i_data)

def X_field_raw(x_field_data, *args):
    return x_field_data

def Y_field_raw(y_field_data, *args):
    return y_field_data

def Z_field_raw(z_field_data, *args):
    return z_field_data
def H_set_raw(h_set_data, *args):
    return h_set_data, *args
def dR_dH_raw(r_raw, field_set, *args):
    return diffs(field_set, r_raw)
def dG_dH_raw(g_raw, field_set, *args):
    return diffs(field_set, g_raw)
def dI_raw(i_raw, *args):
    return diffIV(i_raw)
def dV_raw(v_raw, *args):
    return diffIV(v_raw)
def dI_dH_raw(i_raw, field_set, *args):
    return diffs(field_set, i_raw)
def dV_dH_raw(v_raw, field_set, *args):
    return diffs(field_set, v_raw)
def NdI_raw(i_raw, *args):
    return NormalizedDiff(i_raw)
def SPdI_raw(i_raw, field_data, *args):
    return SlopeDiff(i_raw, field_data)
def HdIS_raw(dr_dH_raw, r_raw, *args):
    return HdIScount(dr_dH_raw, r_raw)
def HdR_raw(field_set, r_raw, *args):
    return diffs(field_set, r_raw)
def HdG_raw(field_set, g_raw, *args):
    return diffs(field_set, g_raw)
def dR_raw(r_raw, *args):
    return diffIV(r_raw)
def dG_raw(g_raw, *args):
    return diffIV(g_raw)
def NdR_raw(r_raw, *args):
    return NormalizedDiff(r_raw)
def NdG_raw(g_raw, *args):
    return NormalizedDiff(g_raw)
def HdRS_raw(r_Hdr, current, *args):
    return HdIScount(r_Hdr, current)
def HdGS_raw(g_Hdg, voltage, *args):
    return HdIScount(g_Hdg, voltage)
def time_raw(time, *args):
    return time
def f_raw(f, *args):
    return f
def I_raw(i, *args):
    return i
def G_raw(i_data,v_data, *args):
     return division(i_data, v_data)


function_list = ['NLI','NLV', 'HSI', 'diffHDC', 'diffJ', 'NdiffJ', 'I_min_max','V_min_max', 'dFitI','dFitV', 'dfiI', 'HSR', 'HSG', 'dFitR', 'dFitG', 'R_min_max_0','G_min_max_0', 'dR/dI', 'dG/dV', 'FitG', 'FitR', 'TdI', 'FTIt']
raw_data_list = [ 'V (V)',
                  'I (A)',
                  'R (ohm)',
                  'G',
                  'X field (Oe)',
                  'Y field (Oe)',
                  'Z field (Oe)',
                  'Hset (Oe)',
                  'dR/dH',
                  'dG/dH',
                  'dI',
                  'dV',
                  'dI/dH',
                  'dV/dH',
                  'NdI',
                  'SPdI',
                  'HdIS',
                  'HdR',
                  'HdG',
                  'dR',
                  'dG',
                  'NdR',
                  'NdG',
                  'HdRS',
                  'HdGS']

In [ ]:
### Przypisanie nagłówków do funkcji ########
pd.set_option('mode.chained_assignment', None)
conn = {
        'V (V)': 'V_raw(data[all_files[i]][V])',
        'I (A)': 'I_raw(data[all_files[i]][I])',
        'R (ohm)': 'R_raw(data[all_files[i]][V], data[all_files[i]][I])',
        'G' : 'G_raw(data[all_files[i]][I], data[all_files[i]][V])',
        'X field (Oe)': 'X_field_raw(data[all_files[i]][X_field])',
        'Y field (Oe)': 'Y_field_raw(data[all_files[i]][Y_field])',
        'Z field (Oe)': 'Z_field_raw(data[all_files[i]][Z_field])',
        'Hset (Oe)' : 'H_set_raw(data[all_files[i]][H_set])',
        'dR/dH': 'dR_dH_raw(data[all_files[i]][R], data[all_files[i]][H_set])',
        'dG/dH': 'dG_dH_raw(data[all_files[i]][G], data[all_files[i]][H_set])',
        'dI': 'dI_raw(data[all_files[i]][I])',
        'dV' : 'dV_raw(data[all_files[i]][V])',
        'dI/dH' : 'dI_dH_raw(data[all_files[i]][I], data[all_files[i]][H_set])',
        'dV/dH': 'dV_dH_raw(data[all_files[i]][V], data[all_files[i]][H_set])',
        'NdI' : 'NdI_raw(data[all_files[i]][I])',
        'SPdI': 'SPdI_raw(data[all_files[i]][I], data[all_files[i]][H_set])',
        'HdIS' : "HdIS_raw(data[all_files[i]][dI_dH], data[all_files[i]][R])",
        'HdR' : 'HdR_raw(data[all_files[i]][H_set], data[all_files[i]][R])',
        'HdG' : 'HdG_raw(data[all_files[i]][H_set], data[all_files[i]][G])',
        'dR' : 'dR_raw(data[all_files[i]][R])',
        'dG' : 'dG_raw(data[all_files[i]][G])',
        'NdR' : 'NdR_raw(data[all_files[i]][R])',
        'NdG' : 'NdG_raw(data[all_files[i]][G])',
        'HdRS' : 'HdRS_raw(data[all_files[i]][HdR], data[all_files[i]][I])',
        'HdGS': 'HdGS_raw(data[all_files[i]][HdG], data[all_files[i]][V])',

}

def add_columns():
  ##### Dodawanie brakujących kolumn ########
  for i in range (0, len(all_files)):
    header =  data[all_files[i]].columns.values
    for head in raw_data_list:
      if head in header:
        continue
      else:
        data[all_files[i]][head] = np.nan
        fun = conn[head]
        result = eval(fun)
        for row in range(len(data[all_files[i]][head])):
          try:
            data[all_files[i]][head][row] = result[row]
          except Exception as e:
            continue
        data[all_files[i]][head].astype('float')
        logger.info("Added column: " + head + " to " + all_files[i])
        # except:
        #   data[all_files[i]][head] = np.nan
        #   logger.info("Added column: " + head + " to " + all_files[i]+"with error")
  return data

In [ ]:
def function_pik(k, *args):
  for chart in file_picker.value:
    match k:
      case 'NLI':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0], I:results[1]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y=I).mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )

      case 'NLV':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][V].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0], V:results[1]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y=V).mark_point(size=10).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'HSI':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0], I:results[1]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y=I).mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'diffHDC':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0], 'diffH_{DC}':results[1]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y='diffH_{DC}').mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'diffJ':
        try:
          results = eval(f'{w.children[0].value}'+f'({area_field.children[0].value },{data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0], I:results[1]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y=I).mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'NdiffJ':
        try:
          results = eval(f'{w.children[0].value}'+f'({area_field.children[0].value },{data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0], I:results[1]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y=I).mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )

      case 'I_min_max':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          print(str(chart)+":\n"+"Minimal I(A):" + str(results[0])+f"\n"+"Maximal I(A):"+str(results[1])+"\n")
          return
        except Exception as error:
          print("Chart could not be displayed due to error:", error )

      case 'V_min_max':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][V].to_list()},{data[chart][H_set].to_list()})')
          print(str(chart)+":\n"+"Minimal V(V):" + str(results[0])+f"\n"+"Maximal V(V):"+str(results[1])+"\n")
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dFitI':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][dI].dropna().to_list()},{data[chart][H_set].to_list()[0:len(data[chart][dI].dropna().to_list())]},{int(l.children[0].value)}, {int(p.children[0].value)})')
          pf = pd.DataFrame({H_set:results[0], 'dFitI':results[1]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y='dFitI').mark_line(color = "red").properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dFitV':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][dV].dropna().to_list()},{data[chart][H_set].to_list()[0:len(data[chart][dV].dropna().to_list())]},{int(l.children[0].value)}, {int(p.children[0].value)})')
          pf = pd.DataFrame({H_set:results[0], 'dFitV':results[1]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y='dFitV').mark_line(color = "red").properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dfiI':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0], I:results[1]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y=I).mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )

      case 'HSR':  #TO DO
        pass
      case 'HSG':   #TO DO
        pass
      case 'dFitR':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][dR].dropna().to_list()},{data[chart][H_set].to_list()[0:len(data[chart][dR].dropna().to_list())]},{int(l.children[0].value)}, {int(p.children[0].value)})')
          pf = pd.DataFrame({H_set:results[0], 'dFitR':results[1]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y='dFitR').mark_line(color = "red").properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dFitG':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][dG].dropna().to_list()},{data[chart][H_set].to_list()[0:len(data[chart][dG].dropna().to_list())]},{int(l.children[0].value)}, {int(p.children[0].value)})')
          pf = pd.DataFrame({H_set:results[0], 'dFitG':results[1]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y='dFitG').mark_line(color = "red").properties(title = chart).interactive() )
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'R_min_max_0':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][R].to_list()},{data[chart][H_set].to_list()})')
          if len(results) ==2:
            print(str(chart)+":\n"+"Minimal R(Ohms):" + str(results[0])+f"\n"+"Maximal R(Ohms):"+str(results[1])+"\n")
          else:
            print(str(chart)+":\n"+"Minimal R(Ohms):" + str(results[0])+f"\n"+"Maximal R(Ohms):"+str(results[1])+"\n"+ "Zero H:" + str(results[2])+"\n")

        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'G_min_max_0':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][G].to_list()},{data[chart][H_set].to_list()})')
          if len(results) ==2:
            print(str(chart)+":\n"+"Minimal G(1/Ohms):" + str(results[0])+f"\n"+"Maximal G(1/Ohms):"+str(results[1])+"\n")
          else:
            print(str(chart)+":\n"+"Minimal G(1/Ohms):" + str(results[0])+f"\n"+"Maximal G(1/Ohms):"+str(results[1])+"\n"+ "Zero H:" + str(results[2])+"\n")
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dR/dI':
        try:
          results = eval(f'd2R'+f'({data[chart][dR].dropna().to_list()},{data[chart][dI].dropna().to_list()},{data[chart][H_set].to_list()[0:len(data[chart][dR].dropna().to_list())]} )')
          pf = pd.DataFrame({'dRdI':results[1], H_set:results[0]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y='dRdI').mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dG/dV':
        try:
          results = eval(f'd2G'+f'({data[chart][dG].dropna().to_list()},{data[chart][dV].dropna().to_list()},{data[chart][H_set].to_list()[0:len(data[chart][dG].dropna().to_list())]} )')
          pf = pd.DataFrame({'dGdV':results[1], H_set:results[0]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=H_set, y='dGdV').mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'FitG':
        try:
          results = FitG(data[chart][H_set].to_list(),data[chart][G].to_list(),int(l.children[0].value), int(p.children[0].value))
          pf = pd.DataFrame({H_set:results[0], 'FitG':results[1]})
          print("g0:"+f"{results[2][0]}"+"\n"+"g1:"+f"{results[2][1]}"+"\n"+"g2:"+f"{results[2][2]}"+"\n")
          display(alt.Chart(data[chart], width = 400, height = 300).encode(x=H_set, y=G).mark_point().interactive()+alt.Chart(pf).encode(x=H_set, y='FitG').mark_line(color = "red").properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'FitR':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][H_set].to_list()},{data[chart][R].to_list()},{int(l.children[0].value)}, {int(p.children[0].value)})')
          pf = pd.DataFrame({H_set:results[0], 'FitR':results[1]})
          print("g0:"+f"{results[2][0]}"+"\n"+"g1:"+f"{results[2][1]}"+"\n"+"g2:"+f"{results[2][2]}"+"\n")
          if args[0] == True:
            display(alt.Chart(data[chart], width = 400, height = 300).encode(x=H_set, y=R).mark_point().interactive() + alt.Chart(pf).encode(x=H_set, y='FitR').mark_line(color = "red").properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'TdI':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][time].to_list()},{data[chart][I].to_list()},{data[chart][AHac].to_list()} )')
          pf = pd.DataFrame({'TdI':results[1], AHac:results[0]})
          if args[0] == True:
            display(alt.Chart(pf, width = 400, height = 300).encode(x=AHac, y='TdI').mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'FTIt':
        try:
          sampling_rate = 2
          results1 = eval(f'TdI'+f'({data[chart][time].to_list()},{data[chart][I].to_list()},{data[chart][AHac].to_list()} )')
          pf1 = pd.DataFrame({'TdI':results1[1], AHac:results1[0]})

          results2 = eval(f'{w.children[0].value}'+f'({pf1["TdI"]}, {sampling_rate})')
          pf2 = pd.DataFrame({f:results2[0], 'FTIt':results2[1]})
          if args[0] == True:
            display(alt.Chart(pf2, width = 400, height = 300).encode(x=f, y='FTIt').mark_point(size=1).properties(title = chart).interactive())
          return pf2
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case _ :
        pass

def online_analysis(w,m,*args):
  for chart in file_picker_online.value:
    i = 0
    for mp in [w,m]:

      try:
          if i == 0:
            x_data= data[chart][w].to_list()
          if i ==1:
            y_data = data[chart][m].to_list()
      except Exception as error:
        print("Chart could not be displayed due to error:", error )

      i += 1
    results = pd.DataFrame({w:x_data, m:y_data})
    display(alt.Chart(results).encode(x=w, y=m).mark_point(size=1).properties(title = chart).interactive())



In [ ]:
data = read_files()[0]
all_files = read_files()[1]

INFO:LOG::Files loaded:0
INFO:LOG::Files loaded:0


None
None


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout
try:
  file_picker_online = widgets.SelectMultiple(
        options=list(all_files),
        value= [all_files[0]],
        description='Files',
        enabled = False)
  output1 = widgets.Output()
  def f(x):
      output1.clear_output()
      p = ll.children[0].value
      t = kk.children[0].value
      with output1:
        online_analysis(p, t, True)
      return x

  ll = interactive(f, x = widgets.Dropdown(
      options=raw_data_list,
      description='X:',
      disabled=False,
  ))
  kk = interactive(f, x = widgets.Dropdown(
      options=raw_data_list,
      description='Y:',
      disabled=False,
  ))

except Exception as k:
  print(k)

def main2():
  try:


  # button = widgets.Button(description="Generate graph")




    # display(w)
    # display(l)
    # display(k)
    # button.on_click(ff())
    # title = widgets.Label(value="The $m$ in $E=mc^2$:")
    # title.style.font_size = "100px"

    grid2 = GridspecLayout(6, 1)
    grid2[0, 0] = ll

    grid2[2, 0] = kk

    grid2[4, 0] = file_picker_online
    # grid[4, 0] = button



  except IndexError:
    print("Twój folder jest pusty!")
  except NameError:
    print('Zanimu uruchomisz program wczytaj dane!')
  return AppLayout(header=None,
              left_sidebar=grid2,
              center=output1,
              right_sidebar=None,
              footer=None)

list index out of range


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout
try:
  file_picker = widgets.SelectMultiple(
        options=list(all_files),
        value= [all_files[0]],
        description='Files',
        enabled = False)
  output = widgets.Output()
  def f(x):
      output.clear_output()
      k = w.children[0].value
      with output:
        function_pik(k, True)
      return x

  w = interactive(f, x = widgets.Dropdown(
      options=function_list,
      description='Function list:',
      disabled=False,
  ))
  l = interactive(f,x = widgets.BoundedIntText(
      value=0,
      step=1,
      max = 100000000,
      description='Start fit point:',
      disabled=False
  ))
  p = interactive(f,x = widgets.BoundedIntText(
      value=0,
      step=1,
      max = 100000000,
      description='Stop fit point:',
      disabled=False
  ))

  area_field = interactive(f,x = widgets.BoundedFloatText(
      value=0.000001,
      min=0,
      description='Area:',
      disabled=False
  ))
except Exception as k:
  print(k)

def main():
  try:


  # button = widgets.Button(description="Generate graph")




    # display(w)
    # display(l)
    # display(k)
    # button.on_click(ff())
    # title = widgets.Label(value="The $m$ in $E=mc^2$:")
    # title.style.font_size = "100px"

    grid = GridspecLayout(6, 1)
    grid[0, 0] = w
    grid[1, 0] = l
    grid[2, 0] = p
    grid[3, 0] = area_field
    grid[4, 0] = file_picker
    # grid[4, 0] = button



  except IndexError:
    print("Twój folder jest pusty!")
  except NameError:
    print('Zanimu uruchomisz program wczytaj dane!')
  return AppLayout(header=None,
              left_sidebar=grid,
              center=output,
              right_sidebar=None,
              footer=None)

list index out of range


# **Data parser**

Program, który zmienia nagłówki w plikach na takie, które obsługuje program a następnie zapisuje na dysku nowe pliki ze zmienionymi nagłówkami. **Uruchamiamy tylko raz dla danego zestawu plików.** W przypadku uruchomienia parsera, po zakończeniu działania konieczne jest ponowne uruchomienie **Backgroud**

In [ ]:
data_parser()

time (s)
V (V)
f (Hz)
AHac (Oe)
Vsense (V)
Vbias (V)
I (A)
Phase
R (ohm)
dI/dH
dI
dV
X field (Oe)
Y field (Oe)
Z field (Oe)
Hset (Oe)
G
I/Phase
I/Ax
dG
dR
dI/dV
dV/dI
dG/dH
NdR
NdG
NdI
HdIS
dV/dH
SPdI
HdIS
HdRS
HdGS
G(t)
R(t)
index
Enter new header for H in /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv: Hset (Oe)
Enter new header for I in /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv: I (A)
Enter new header for V in /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv: V (V)
Enter new header for indx in /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv: index
Created file: /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv_headers_ok


#Dodawanie kolumn
Kod dodaje brakujące kolumny z wynikami analizy "online".

In [ ]:
data = add_columns()

INFO:LOG::Added column: R (ohm) to /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv_headers_ok
INFO:LOG::Added column: G to /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv_headers_ok
INFO:LOG::Added column: X field (Oe) to /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv_headers_ok
INFO:LOG::Added column: Y field (Oe) to /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv_headers_ok
INFO:LOG::Added column: Z field (Oe) to /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv_headers_ok
INFO:LOG::Added column: dR/dH to /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv_headers_ok
INFO:LOG::Added column: dG/dH to /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv_headers_ok
INFO:LOG::Added column: dI to /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv_headers_ok
INFO:LOG::Added column: dV to /gdrive/MyDrive/AGH/wisniowski/test_data2/dane_testowe_4.csv_headers_ok
INFO:LOG::Added column: dI/dH to /gdrive/M

#Wizualizacja

Analizy offline

In [ ]:
main()

AppLayout(children=(GridspecLayout(children=(interactive(children=(Dropdown(description='Function list:', opti…

Analizy online

In [ ]:
main2()

AppLayout(children=(GridspecLayout(children=(interactive(children=(Dropdown(description='X:', options=('V (V)'…

# Zapis do pliku

In [ ]:
########## Save Choosen Data ###########

function_pik(w.children[0].value, False).to_csv(os.path.basename(src_path+file_picker.value[0]).split("/")[0][:-15]+"_"+w.children[0].value+".csv", index=False)
